# Feature selection and intial predictions

In [ ]:
%reload_ext autoreload
%autoreload 2
import os 
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import plotly 
import plotly.graph_objects as go
from datetime import timedelta

sys.path.insert(0, "./../../")

# from src.preprocessing.feature_preparation import prepare_features_for_season_based_predictions
from src.preprocessing.DataPreparator import DataPreparator

In [ ]:
data_p = DataPreparator()
all_games = data_p.get_games_df()

# home team wins in 60% of the cases
display(all_games["home_win"].mean())

In [ ]:
import pickle
# for the actual train set all_games.iloc[:round(len(all_games.index) * 0.8)]["home_win"].mean()
display(all_games)

with open("tmp_f_file.pickle", "wb") as file:
        pickle.dump(all_games, file)

['home_recent_team_rebound', 'home_recent_team_turnover', 'home_recent_subs', 'home_recent_timeout', 'home_recent_jump_balls_won', 'home_recent_ejection', 'home_recent_team_ejection', 'home_recent_scoring_leader', 'home_recent_scoring_leader_points', 'home_recent_made_max_shot_distance', 'home_recent_made_min_shot_distance', 'home_recent_made_mean_shot_distance'] not in index"

Base line for predictions is 0.597 -> which is ~60%, that means if we predict that each game will be won by the home team we are correct in 60% of cases. Any improvements over 60% are actual benefits of our models.

In [ ]:
x_train, y_train, x_test, y_test = data_p.prepare_data_splits("2017-2018")
display(x_train[:3], len(x_train))
display(y_train[:3], len(y_train))

In [ ]:
len(x_train[0])

# Feature selection
Select and transform only neccesary features for the model.
   
https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b

In [ ]:
cor = all_games.corr()
home_win_s = cor["home_win"].abs().sort_values(ascending=False)
display(home_win_s[:5])

fig = go.Figure()
fig.add_trace(go.Scatter(x=home_win_s, y=home_win_s.index))

# Still need to test and determine best features


## Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

# max_depth=100
r_forest = RandomForestRegressor(n_estimators=150, criterion='squared_error', 
                                 random_state=42, max_depth=13)
r_forest.fit(x_train, y_train)

# evaluate on train data set ->> overfitting but just to see the accuracy
print("Starting predictions ...")
y_pred_train = []
for row in x_train:
    y_pred_train.append(round(r_forest.predict([row])[0]))
    
print(f"Accuracy on the train set is: {accuracy_score(y_train, y_pred_train, normalize=True) * 100} %")

Evaluation on the train set yield an accuracy of 100% which means model can overfit to the given data correctly!

In [ ]:
print("Evaluating on the test data set!")

y_pred_test = []
for row in x_test:
    y_pred_test.append(round(r_forest.predict([row])[0]))
    
print(f"Accuracy on the train set is: {accuracy_score(y_test, y_pred_test, normalize=True) * 100} %")

For the "wrong" data points which encapsulate future data the model cchieves around 83% prediction accuracy on the test data set when the split is of size 0.8, 0.1 and 0.1. And 82% if the data is lowered to 82%.
For the right data points, the features of which are based on a running average of values in the previous games the model achives accuracy from 62 to 72% which is 20% better than the baseline prediction.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

precision, recall, f1, _s = precision_recall_fscore_support(y_test, y_pred_test, average='macro')
print(precision, recall, f1)

cmp_ht = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_test))
fig, ax = plt.subplots(figsize=(15,15))
plt.rc("font", size=30)
cmp_ht.plot(ax=ax)
plt.savefig("conf_matrix_rf.png")

## SVM
Evaluating the features on SVM which should be home suited to this use case

In [ ]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
    
# Features should be scaled before using SVM!
# ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’
svm_p = svm.SVC(kernel="linear", C=0.5, random_state=42)
svm_p.fit(x_train, y_train)

# evaluate on train data set ->> overfitting but just to see the accuracy
print("Starting predictions ...")
y_pred_train = []
for row in x_train:
    y_pred_train.append(round(svm_p.predict([row])[0]))
    
print(f"Accuracy on the train set is: {accuracy_score(y_train, y_pred_train, normalize=True) * 100} %")

In [ ]:
print("Evaluating on the test data set!")

y_svm_pred_test = []
for row in x_test:
    y_svm_pred_test.append(round(svm_p.predict([row])[0]))
    
print(f"Accuracy on the train set is: {accuracy_score(y_test, y_svm_pred_test, normalize=True) * 100} %")

In [ ]:
precision, recall, f1, _s = precision_recall_fscore_support(y_test, y_svm_pred_test, average='macro')
print(precision, recall, f1)

cmp_ht = ConfusionMatrixDisplay(confusion_matrix(y_test, y_svm_pred_test))
fig, ax = plt.subplots(figsize=(10,10))
plt.rc("font", size=30)
cmp_ht.plot(ax=ax)
plt.savefig("conf_matrix_svm.png")